In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import re
from datetime import datetime

class NuGoldChatbot:
    def __init__(self, root):
        self.root = root
        self.root.title("Nu-Gold Customer Support")
        self.root.state('zoomed')  
        
        # Configure colors with earthy tones
        
        self.root.configure(bg="#F5E4A3")  # Light gold for main background
        self.chat_bg = "#D4AF37"           # Metallic gold for chat background
        self.user_bg = "#E8C78C"           # Pale gold for user messages
        self.bot_bg = "#E6D5A8"            # Soft gold for bot messages
        self.button_bg = "#B8860B"         # Dark gold for buttons
        self.button_fg = "#333333"         # Dark charcoal for button text
        self.nugold_color = "#b8860b"      # Nugold for accents
        
        # Create GUI elements
        self.create_widgets()
        
        # Initialize responses
        self.initialize_responses()
        
        # Welcome message
        self.add_bot_message("Mhoro! Welcome to Nu-Gold Products Support. How can we assist you today?")

    def create_widgets(self):
        # Header with Nu-Gold branding
        self.header = tk.Label(
            self.root, text="Nu-Gold Traditional Grains Products", 
            font=("Arial", 16, "bold"), bg="#f8f4e6", fg=self.nugold_color
        )
        self.header.pack(pady=(10, 5), fill=tk.X)
        
        # Chat display area
        self.chat_display = scrolledtext.ScrolledText(
            self.root, wrap=tk.WORD, font=("Arial", 12),
            bg=self.chat_bg, state='disabled'
        )
        self.chat_display.pack(padx=10, pady=5, fill=tk.BOTH, expand=True)
        
        # Configure tags
        self.chat_display.tag_config('nugold', foreground=self.nugold_color)
        self.chat_display.tag_config('contact', foreground="#8b0000", font=("Arial", 12, "bold"))
        self.chat_display.tag_config('urgent', foreground="#ff0000", font=("Arial", 12, "bold"))
        
        # Input frame
        input_frame = tk.Frame(self.root, bg="#f8f4e6")
        input_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # User input field
        self.user_input = tk.Entry(
            input_frame, font=("Arial", 12), width=60
        )
        self.user_input.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.user_input.bind("<Return>", lambda event: self.send_message())
        
        # Send button
        self.send_button = tk.Button(
            input_frame, text="Send", width=10,
            bg=self.button_bg, fg=self.button_fg, font=("Arial", 12),
            command=self.send_message
        )
        self.send_button.pack(side=tk.RIGHT, padx=(10, 0))
        
        # Button frame
        button_frame = tk.Frame(self.root, bg="#f8f4e6")
        button_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Info button
        self.info_button = tk.Button(
            button_frame, text="Product Info", width=15,
            bg=self.button_bg, fg=self.button_fg, font=("Arial", 12),
            command=self.show_product_info
        )
        self.info_button.pack(side=tk.LEFT, padx=(10, 5))
        
        # Help button
        self.help_button = tk.Button(
            button_frame, text="Customer Service", width=15,
            bg="#8b0000", fg="#ffffff", font=("Arial", 12),
            command=self.show_customer_service
        )
        self.help_button.pack(side=tk.LEFT, padx=5)
        
        # Exit button
        self.exit_button = tk.Button(
            button_frame, text="Exit", width=15,
            bg="#8b0000", fg="#ffffff", font=("Arial", 12),
            command=self.exit_chat
        )
        self.exit_button.pack(side=tk.RIGHT, padx=(5, 10))

    def initialize_responses(self):
        self.responses = {
            # Greetings
            "greeting": [
                (r"\b(hi|hello|mhoro|greetings)\b", "Mhoro! Welcome to Nu-Gold Products Support. How can we assist you today?"),
                (r"\b(good morning|morning)\b", "Good morning! Welcome to Nu-Gold support. How may we help you?"),
                (r"\b(good afternoon|afternoon)\b", "Good afternoon! Welcome to Nu-Gold support. How may we help you?"),
                (r"\b(good evening|evening)\b", "Good evening! Welcome to Nu-Gold support. How may we help you?")
            ],
            
            # Product Information
            "products": [
                (r"\b(products|range|offer|what do you sell)\b", self.get_product_range),
                (r"\b(bread|confectionery)\b", "Our Nu-Gold bread and confectionery:\n- Made with sweet potatoes flour and traditional grains (millet, sorghum)\n- Healthier alternative to wheat bread\n- Various traditional snacks available"),
                (r"\b(oils|edible oils|cooking oil)\b", "Nu-Gold edible oils:\n- Cold-pressed from traditional grains\n- Rich in essential fatty acids\n- Perfect for Zimbabwean cooking methods"),
                (r"\b(feed|stock feed|animal feed)\b", "Nu-Gold stock feed:\n- Nutritionally balanced for livestock\n- Formulated with local grain by-products\n- Produced at UZ Agro-Industrial Park"),
                (r"\b(snacks|puffed snacks)\b", "Nu-Gold puffed snacks:\n- Made from 100% traditional grains\n- No artificial preservatives\n- Supporting local farmers"),
                (r"\b(grains|traditional grains|millet|sorghum|rapoko)\b", "We promote these indigenous grains:\n- Pearl millet (mhunga)\n- Sorghum (mapfunde)\n- Finger millet (zviyo)\n- Rapoko (rukweza)\nAll drought-resistant and nutrient-rich")
            ],
            
            # About Nu-Gold
            "about": [
                (r"\b(about nu.?gold|information|initiative)\b", self.get_nugold_intro),
                (r"\b(university of zimbabwe|uz|agro.?industrial park)\b", "Nu-Gold is proudly produced by:\n- University of Zimbabwe\n- At the Agro-Industrial Park & Innovation Hub\n- Supporting Vision 2030 through local innovation"),
                (r"\b(benefits|importance|why nugold)\b", "Nu-Gold contributes to:\n- National food and nutrition security\n- Preservation of traditional grains\n- Local employment creation\n- Farmer empowerment\n- Import substitution")
            ],
            
            # Locations and Availability
            "availability": [
                (r"\b(where to buy|purchase|available|outlets|stockists)\b", "Nu-Gold products are available at:\n- UZ Agro-Industrial Park (Mt Pleasant)\n- Selected retailers nationwide\nCall 0776277714 for nearest stockist"),
                (r"\b(harare|bulawayo|mutare|masvingo|gweru)\b", "We're expanding across Zimbabwe! Currently available in major cities:\n- Harare\n- Bulawayo\n- Mutare\n- Masvingo\n- Gweru\nCall 0776277714 for specific store locations")
            ],
            
            # Support
            "support": [
                (r"\b(contact|number|phone|call|customer service|help|support)\b", self.get_customer_service_response),
                (r"\b(complaint|issue|problem|wrong|broken|damaged)\b", "We sincerely apologize for this experience.\n\nFor immediate assistance:\nCALL our Customer Service Hotline: 0776277714\nEMAIL: nugold@uz.ac.zw\n\nPlease have your product details ready."),
                (r"\b(hours|open|time|when are you available)\b", "Customer support hours:\n- Monday-Friday: 8:00AM to 4:30PM\n- After hours: Leave message at 0776277714\n- Physical visits by appointment only")
            ],
            
            # Closing
            "closing": [
                (r"\b(bye|goodbye|end|farewell)\b", "Thank you for supporting our local products!\nFor further help, call 0776277714.\nTatenda zvikuru!"),
                (r"\b(thanks|thank you|ndatenda)\b", "Tatenda! Remember our contact:\n0776277714\nIs there anything else about Nu-Gold products we can help with?")
            ]
        }

    def get_customer_service_response(self):
        return ("NU-GOLD CUSTOMER SERVICE\n\n"
                "For immediate assistance, please contact:\n\n"
                "HOTLINE: 0776277714\n"
                "EMAIL: nugold@uz.ac.zw\n"
                "VISIT: UZ Agro-Industrial Park, Mt Pleasant\n\n"
                "Our team is ready to help you with:\n"
                "- Product inquiries\n- Complaints\n- Bulk orders\n- Retailer information")

    def show_customer_service(self):
        self.add_bot_message(self.get_customer_service_response())

    def get_nugold_intro(self):
        return ("NU-GOLD PRODUCTS (Launched September 2023)\n\n"
                "Produced by: University of Zimbabwe\n"
                "Location: UZ Agro-Industrial Park, Mt Pleasant\n\n"
                "OUR MISSION:\n"
                "- Promote traditional grains\n"
                "- Enhance food security\n"
                "- Support local farmers\n"
                "- Create nutritious products\n\n"
                "CONTACT DETAILS:\n"
                "0776277714\n"
                "nugold@uz.ac.zw\n"
                "Coming soon: www.nugold.co.zw")

    def get_product_range(self):
        return ("NU-GOLD PRODUCT RANGE:\n\n"
                "1. TRADITIONAL GRAIN BREAD & CONFECTIONERY\n"
                "   - Nutrient-rich breads\n"
                "   - Traditional baked goods\n\n"
                "2. COLD-PRESSED EDIBLE OILS\n"
                "   - High-quality cooking oils\n"
                "   - Retains natural nutrients\n\n"
                "3. ANIMAL STOCK FEED\n"
                "   - Scientifically formulated\n"
                "   - Uses local grain by-products\n\n"
                "4. PUFFED GRAIN SNACKS\n"
                "   - 100% traditional grains\n"
                "   - Healthy snack alternatives\n\n"
                "For detailed specifications, call 0776277714")

    def add_bot_message(self, message):
        self.chat_display.configure(state='normal')
        
        # Remove any remaining unsupported characters
        safe_message = message
        self.chat_display.insert(tk.END, "Nu-Gold: " + safe_message + "\n\n")
            
        self.chat_display.configure(state='disabled')
        self.chat_display.see(tk.END)

    def add_user_message(self, message):
        self.chat_display.configure(state='normal')
        self.chat_display.insert(tk.END, "You: " + message + "\n\n")
        self.chat_display.configure(state='disabled')
        self.chat_display.see(tk.END)

    def send_message(self):
        user_input = self.user_input.get().strip()
        if not user_input:
            return
        
        self.add_user_message(user_input)
        self.user_input.delete(0, tk.END)
        
        response = self.get_bot_response(user_input)
        self.add_bot_message(response)

    def get_bot_response(self, user_input):
        user_input = user_input.lower()
        
        # Check all response categories in order of priority
        for category in ["support", "greeting", "products", "about", "availability", "closing"]:
            for pattern, response in self.responses[category]:
                if re.search(pattern, user_input):
                    if callable(response):
                        return response()
                    return response
        
        # Default response
        return ("I didn't quite understand. You can ask about:\n"
                "- Our traditional grain products\n"
                "- Where to buy Nu-Gold items\n"
                "- Benefits of our products\n"
                "Or click 'Customer Service' for immediate help")

    def show_product_info(self):
        info = """NU-GOLD PRODUCTS INFORMATION

Customer Service: 0776277714

OUR PRODUCT RANGE:
1. TRADITIONAL GRAIN BREADS
   - Made with millet, sorghum
   - High fiber, gluten-free options

2. COLD-PRESSED OILS
   - From traditional grains
   - High smoke point for cooking

3. ANIMAL FEED
   - Formulated by UZ experts
   - Uses local ingredients

4. PUFFED SNACKS
   - 100% traditional grains
   - No artificial additives

PRODUCED AT:
- University of Zimbabwe
- Agro-Industrial Park
- Call 0776277714"""
        messagebox.showinfo("Nu-Gold Products", info)

    def exit_chat(self):
        if messagebox.askyesno("Exit", "Are you sure you want to end the chat?"):
            self.add_bot_message("Thank you for contacting Nu-Gold support!\nRemember our number: 0776277714\nTatenda nekuonana!")
            self.root.after(1000, self.root.destroy())

# Run the chatbot
if __name__ == "__main__":
    root = tk.Tk()
    chatbot = NuGoldChatbot(root)
    root.mainloop()

Questions To Ask:

**1. Product Information:**
- What products does Nu-Gold offer?
- Tell me about your traditional grain bread
- What types of cooking oils do you produce?
- How is your animal feed different from others?
- What flavors of puffed snacks do you have?
- Which traditional grains do you use in your products?
- Are Nu-Gold products gluten-free?
- What are the health benefits of your bread?

**2. Availability & Where to Buy:**
- Where can I buy Nu-Gold products in Harare?
- Do you have stockists in Bulawayo?
- Which shops sell Nu-Gold in Mutare?
- Can I buy directly from your factory?
- Is Nu-Gold available in supermarkets?
- Where is your nearest outlet to the city center?
- Do you deliver to rural areas?

**3. Customer Service & Support:**
- What's your customer service number?
- How do I make a complaint about a product?
- Who do I contact for bulk orders?
- What are your operating hours?
- My bread was stale when I bought it - what should I do?
- The oil bottle was leaking - can I get a replacement?
- How do I store your products properly?

**4. About Nu-Gold:**
- Who owns Nu-Gold products?
- Where is your production facility located?
- When was Nu-Gold established?
- How does Nu-Gold support local farmers?
- What makes Nu-Gold different from other brands?
- Is Nu-Gold part of the University of Zimbabwe?
- What is your company's mission?

**5. Technical Questions:**
- What's the shelf life of your snacks?
- How should I store your cooking oil?
- Are your products suitable for diabetics?
- Do you use any preservatives in your bread?
- What's the nutritional value of your puffed snacks?
- Are your products certified by any health organizations?

**6. General Questions:**
- Why should I choose traditional grain products?
- How does Nu-Gold contribute to Zimbabwe's economy?
- Are you planning to expand your product range?
- Do you offer any discounts for large orders?
- Can I visit your production facility?
- Do you have any upcoming new products?

These questions are designed to:
1. Test all aspects of your chatbot's knowledge base
2. Cover common customer inquiries
3. Verify the bot can handle different question phrasings
4. Ensure proper responses for both simple and complex questions

The bot should be able to provide satisfactory answers to all of these based on the response dictionary in your code. 